

## 🧠 OVERVIEW: Development Flow

You’ll follow this **logical sequence**:

| Stage                                   | Purpose                          | File Type                 | Tools Used                  |
| --------------------------------------- | -------------------------------- | ------------------------- | --------------------------- |
| 1. Problem definition & data collection | Understand the goal, gather data | `.ipynb`                  | Notebook, Pandas            |
| 2. EDA & preprocessing                  | Clean, explore, and prepare data | `.ipynb`                  | Pandas, Matplotlib, Seaborn |
| 3. Model building & evaluation          | Train and test ML model          | `.ipynb`                  | Scikit-learn                |
| 4. Model saving                         | Save model for reuse             | `.ipynb` → outputs `.pkl` | Joblib/Pickle               |
| 5. Backend app setup                    | Serve model predictions          | `.py`                     | Flask                       |
| 6. Frontend (optional)                  | HTML interface                   | `.html`, `.css`           | HTML/CSS                    |
| 7. Database (optional)                  | Store predictions/logs           | `.py` + DB                | SQLite/MySQL/PostgreSQL     |

---

# 🔹 Step-by-Step Guide

---

## 🥇 Step 1: Create Jupyter Notebook (`model_dev.ipynb`)

### Purpose:

All **experimentation and analysis** happen here.

### Tasks:

1. Import libraries (`pandas`, `numpy`, `matplotlib`, `seaborn`, `sklearn`)
2. Load dataset (`pd.read_csv()` or DB connection)
3. Perform **EDA (Exploratory Data Analysis)**:

   * Check nulls, datatypes, duplicates
   * Visualize relationships (scatter, hist, corr heatmap)
4. Clean & preprocess data (encoding, normalization, imputation)
5. Split data (train-test)
6. Train your ML model (RandomForest, LogisticRegression, etc.)
7. Evaluate (accuracy, r², confusion matrix, etc.)
8. Save model → `model.pkl`

### Code Example:

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import joblib

# Load Data
data = pd.read_csv("house_prices.csv")

# Prepare data
X = data[['area', 'bedrooms', 'bathrooms']]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
print("R2:", r2_score(y_test, model.predict(X_test)))

# Save model
joblib.dump(model, 'model.pkl')
```

✅ **Output file:** `model.pkl` (used later in Flask)

---

## 🥈 Step 2: Create Flask App (`app.py`)

### Purpose:

Serve predictions using your trained model.

### Tasks:

1. Import Flask & joblib
2. Load the saved `model.pkl`
3. Create routes (`/` and `/predict`)
4. Take input from HTML form or API
5. Pass input to model → return prediction

### Code Example:

```python
from flask import Flask, render_template, request
import joblib
import numpy as np

app = Flask(__name__)
model = joblib.load('model.pkl')

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict', methods=['POST'])
def predict():
    area = float(request.form['area'])
    bedrooms = int(request.form['bedrooms'])
    bathrooms = int(request.form['bathrooms'])
    
    pred = model.predict(np.array([[area, bedrooms, bathrooms]]))[0]
    return render_template('home.html', prediction_text=f'Predicted Price: ${pred:,.2f}')

if __name__ == '__main__':
    app.run(debug=True)
```

---

## 🥉 Step 3: Create Frontend (HTML Template)

### Folder Structure:

```
ML_Project/
│
├── model_dev.ipynb
├── model.pkl
├── app.py
├── templates/
│   └── home.html
└── static/
    └── style.css
```

### home.html

```html
<!DOCTYPE html>
<html>
<head>
  <title>House Price Prediction</title>
</head>
<body>
  <h2>Enter Details</h2>
  <form action="/predict" method="post">
    Area: <input type="text" name="area"><br>
    Bedrooms: <input type="text" name="bedrooms"><br>
    Bathrooms: <input type="text" name="bathrooms"><br>
    <input type="submit" value="Predict">
  </form>

  {% if prediction_text %}
    <h3>{{ prediction_text }}</h3>
  {% endif %}
</body>
</html>
```

---

## 🧱 Step 4: Connect Database (Optional but Common)

If you want to:

* **Store predictions**
* **Log user inputs**
* **Retrieve previous results**

You can integrate a **database**.

### Example: Using SQLite (simplest for beginners)

**Install:**

```bash
pip install flask_sqlalchemy
```

### Modify app.py

```python
from flask_sqlalchemy import SQLAlchemy

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///predictions.db'
db = SQLAlchemy(app)

class Prediction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    area = db.Column(db.Float)
    bedrooms = db.Column(db.Integer)
    bathrooms = db.Column(db.Integer)
    predicted_price = db.Column(db.Float)
```

### After prediction:

```python
new_data = Prediction(area=area, bedrooms=bedrooms, bathrooms=bathrooms, predicted_price=pred)
db.session.add(new_data)
db.session.commit()
```

✅ Database stores user inputs + model outputs.

---

## 🧩 Step 5: Test and Debug

1. Run Flask app:

   ```bash
   python app.py
   ```
2. Open browser: [http://127.0.0.1:5000/](http://127.0.0.1:5000/)
3. Enter data → check prediction
4. Check logs / DB entries

---

## ☁️ Step 6: Deploy (Optional Advanced)

Once it works locally, you can deploy:

### Common free options:

* **Render.com** (best for Flask)
* **Railway.app**
* **PythonAnywhere**
* **AWS EC2 / Azure / GCP (advanced)**

### Process:

1. Push project to **GitHub**
2. On Render → create new **Web Service**
3. Set start command:

   ```
   gunicorn app:app
   ```
4. Wait for build → get live URL

---

## 🧭 Summary: What File & When

| Stage  | File Type       | Purpose                       |
| ------ | --------------- | ----------------------------- |
| Step 1 | `.ipynb`        | Data analysis, model training |
| Step 2 | `.pkl`          | Store trained model           |
| Step 3 | `.py`           | Flask backend server          |
| Step 4 | `.html`         | Web UI (user input)           |
| Step 5 | `.db`           | Store logs or user inputs     |
| Step 6 | (GitHub/Render) | Cloud deployment              |

---

## 🧰 Tools in Each Stage

| Phase           | Tools                       |
| --------------- | --------------------------- |
| Data Processing | Pandas, Numpy               |
| Visualization   | Matplotlib, Seaborn         |
| ML Model        | scikit-learn                |
| Saving Model    | Joblib or Pickle            |
| Web App         | Flask                       |
| Database        | SQLite / MySQL / PostgreSQL |
| Deployment      | Render / Railway / AWS      |

---
